<a href="https://colab.research.google.com/github/langroid/langroid/blob/main/examples/docqa/langroid-lancedb-rag-movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval-Augmented Analytics with Langroid + LanceDB


Say you are working with a large dataset of movie ratings. Let's think about
how to answer questions like this:
> What is the highest rated Comedy movie about college students made after 2010?

To answer this kind of question, we need:
- filtering (on genre, year),
- retrieval (semantic/lexical search on 'college students'),
- computation (highest rated), and
- LLM-based generation of the final answer.

Of course, we'd like to automate the filtering and computation steps -- but how?


We could use an LLM to generate a **Query Plan** for this --
provided the underlying data store supports:
- a filtering language "known" to LLMs (like SQL), and
- a computation language "known" to LLMs (like a Pandas dataframe expression).

This is where [LanceDB](https://github.com/lancedb/lancedb) (the default vector-db in Langroid) comes in:
it's a versatile, highly performant, serverless vector-database that
supports all of these functions within the same storage system and API:
- Fast Full-text search (so you can do lexical search in the same store
  where you do vector/semantic-search)
- SQL-like metadata filtering
- Pandas dataframe interop, so you can ingest dataframes and do pandas computations.
**bold text**
Leveraging Langroid's powerful Multi-Agent and tools orchestration, we built a
3-Agent system consisting of:
- Query Planner: Takes a user's query (like the above) and generates a Query Plan as a tool/function
  consisting of: (a) a SQL-like filter, (b) a possibly rephrased query, and (c) an optional Pandas computation.
- A RAG Agent (powered by LanceDB) that executes the query plan combining
  filtering, RAG, lexical search, and optional Pandas computation.
- A Query Plan Critic that examines the Query Plan and the RAG response, and
  suggests improvements to the Query Planner, if any.

This system can answer questions such as the above.
You can try it out in this notebook, with a dataset of
IMDB movie ratings.

If you want to run it as a script, see here:
https://github.com/langroid/langroid-examples/blob/main/examples/docqa/lance-rag-movies.py



#### Install, setup, import

In [1]:
# Silently install, suppress all output (~2-4 mins)
!pip install -q --upgrade langroid &> /dev/null
!pip show langroid

In [2]:
# various unfortunate things that need to be done to
# control colab notebook behavior.

# (a) output width

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# (b) logging related
import logging
logging.basicConfig(level=logging.ERROR)
import warnings
warnings.filterwarnings('ignore')
import logging
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.ERROR)

# (c) allow async ops in colab
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()


In [22]:
import pandas as pd
from langroid.agent.special.doc_chat_agent import DocChatAgentConfig
from langroid.agent.special.lance_doc_chat_agent import LanceDocChatAgent
from langroid.agent.special.lance_rag.lance_rag_task import LanceRAGTaskCreator

from langroid.utils.configuration import settings
from langroid.embedding_models.models import OpenAIEmbeddingsConfig
from langroid.vector_store.lancedb import LanceDBConfig
settings.cache_type = "fakeredis"
settings.notebook = True

#### OpenAI API Key (Needs GPT4-TURBO)

In [4]:
# OpenAI API Key: Enter your key in the dialog box that will show up below
# NOTE: colab often struggles with showing this input box,
# if so, try re-running the above cell and then this one,
# or simply insert your API key in this cell, though it's not ideal.

import os

from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass('Enter your GPT4-Turbo-capable OPENAI_API_KEY key:', stream=None)




#### Get IMDB ratings & descriptions data

In [5]:
# (1) Get the movies dataset

import requests
file_url = "https://raw.githubusercontent.com/langroid/langroid-examples/main/examples/docqa/data/movies/IMDB.csv"
response = requests.get(file_url)
with open('movies.csv', 'wb') as file:
    file.write(response.content)



In [6]:
 print(
        """
        Welcome to the IMDB Movies chatbot!
        This dataset has around 130,000 movie reviews, with these columns:

        movie, genre, runtime, certificate, rating, stars,
        description, votes, director.

        To keep things speedy, we'll restrict the dataset to movies
        of a specific genre that you can choose.
        """
    )

In [7]:
from ipywidgets import Dropdown
genres = [
          "Action",
          "Adventure",
          "Biography",
          "Comedy",
          "Crime",
          "Documentary",
          "Drama",
          "Fantasy",
          "History",
          "Horror",
          "Music",
          "Musical",
          "Mystery",
          "Romance",
          "Sci-Fi",
          "Sport",
          "Thriller",
          "War",
          "Western",
      ]
dropdown = Dropdown(options=genres, value=genres[0], description="Choose a genre:", disabled=False)
display(dropdown)
genre = dropdown.value

In [8]:
# READ IN AND CLEAN THE DATA
import pandas as pd
df = pd.read_csv("movies.csv")

def clean_votes(value):
    """Clean the votes column"""
    # Remove commas and convert to integer, if fails return 0
    try:
        return int(value.replace(",", ""))
    except ValueError:
        return 0

# Clean the 'votes' column
df["votes"] = df["votes"].fillna("0").apply(clean_votes)

# Clean the 'rating' column
df["rating"] = df["rating"].fillna(0.0).astype(float)

# Replace missing values in all other columns with '??'
df.fillna("??", inplace=True)
df["description"].replace("", "unknown", inplace=True)

# get the rows with selected genre
df = df[df["genre"].str.contains(genre)]

print(
    f"""
[blue]There are {df.shape[0]} movies in {genre} genre, hang on while I load them...
"""
)
# sample 1000 rows for faster testing
df = df.sample(1000)

#### Set up LanceDB Vector-DB and LanceDocChatAgent

In [17]:
# Config LanceDB vector database
import shutil
db_dir = ".lancedb/data"
shutil.rmtree(db_dir)
ldb_cfg = LanceDBConfig(
    collection_name="chat-lance-imdb",
    replace_collection=True,
    storage_path=db_dir,
    embedding=OpenAIEmbeddingsConfig()
)

In [18]:
# configure, create LanceDocChatAgent
cfg = DocChatAgentConfig(
        vecdb=ldb_cfg,
        show_stats=False,
        add_fields_to_content=["movie", "genre", "certificate", "stars", "rating"],
        filter_fields=["genre", "certificate", "rating"],
    )
agent = LanceDocChatAgent(cfg)


#### Ingest data into LanceDocChatAgent

In [19]:
# Ingest the data into LanceDocChatAgent
agent.ingest_dataframe(df, content="description", metadata=[])
df_description = agent.df_description

# inform user about the df_description, in blue
print(
    f"""
Here's a description of the DataFrame that was ingested:
{df_description}
"""
)

### Create, run a 3-agent system to handle user queries


In [ ]:
task = LanceRAGTaskCreator.new(agent, interactive=True)

task.run("Can you help with some questions about these movies?")